# **Problem Statement**

## **Task**
1. Identify all shops that are deemed to have conducted order brushing.
2. For each shop that is identified to have conducted order brushing, identify the
buyers suspected to have conducted order brushing for that shop.

For the purpose of this question, shops are deemed to have conducted order brushing if
their concentrate rate is greater than or equal to 3 at any instance

***Concentrate rate = Number of Orders within 1 hour / Number of Unique Buyers within 1 hour***

For the purpose of this question, suspicious buyers are deemed as the buyer that
contributed the highest proportion of orders to a shop that is deemed to have conducted
order brushing.
For calculation of the highest proportion of orders to a shop, only include the orders that
occured in instances when order brushing has been deemed to have taken place.
In the case where multiple users share the same highest proportion of orders for a
specific shop, all those users are deemed to be suspicious buyers.

## **Approach**

The basic idea is calculating Concentrate rate for all possible 1 hour windows. Here
illustrate the strategy:

a. Select a 1 hour window and take a slice from order data.
Note when you set the time condition, both ends are included in the 1 hour
window. (Hint: For Python, datetime module and timedelta function can be
applied)

b. Find shops which satisfy CR>=3.
For the slice of order data you select, you can use an aggregation function like
groupby to calculate order number and unique user number for each shop in this
1 hour window. And then filter shops which satisfy the CR>=3 condition. They are
brushing shops.

c. Find the userid who conducts order brushing in brushing shops.
When brushing shops are identified, aggregation function can be applied again to
calculate which userid contribute the most orders for each shop. Note if two
userid both have the highest order number, then they are both the answer.

d. Repeat the above procedure for all possible 1 hour window.
For example:

2019-12-27 00:00:00 ~ 2019-12-27 01:00:00,

2019-12-27 00:00:01 ~ 2019-12-27 01:00:01, ...

## **Note**

When you follow the above steps, you may find d step takes a long time since the time
period in our data crosses 5 days and the time window is detailed to second. To reduce
the time consumption, some advanced strategy might be applied. For example:
*   Multiprocessing to brutally calculate them all.
*   Avoid redundant calculation by detecting windows which cover exactly the same orders.

# **Code Time**

First of all, import Pandas and Datetime

In [1]:
import pandas as pd
from datetime import datetime

Initial display of the datasets

In [2]:
train = pd.read_csv('datasets/order_brush_order.csv')
train

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
...,...,...,...,...
222745,31245479170194,110868129,193218997,2019-12-28 23:17:59
222746,31230440360374,91639906,3541807,2019-12-28 19:07:20
222747,31191471145838,29391773,135908070,2019-12-28 08:17:52
222748,31198471732255,61556313,27306783,2019-12-28 10:14:31


Let's have a look on the small dataset first and convert the event_time to seconds

In [3]:
# Set 1 January 2000 as a benchmark, you are free to set any early enough benchmark
time_zero = pd.to_datetime('2000-01-01')

# Create a new column unix_time that contains the total seconds from event_time to the benchmark time
train["event_time"] = pd.to_datetime(train["event_time"]).apply(lambda x: int((x - time_zero).total_seconds()))

# Display
train.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,630721383
1,31118059853484,156423439,46057927,630762860
2,31123355095755,173699291,67341739,630768155
3,31122059872723,63674025,149380322,630766860
4,31117075665123,127249066,149493217,630761875


We might want to sort the orders by time, so let's sort them

In [4]:
train = train.sort_values("event_time")

# Display, they're all now sorted by event_time ascending
train

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,630720000
25985,31075200506752,104804492,97707522,630720000
81020,31075200506753,8715449,97707522,630720000
146598,31075201870570,190969466,170182475,630720002
80355,31075205798264,2859407,12532131,630720005
...,...,...,...,...
114113,31507183252446,149254894,193333760,631151983
206585,31507187390691,147941492,40258063,631151987
126212,31507191066628,187123853,2338306,631151991
160657,31507191066627,154074176,2338306,631151991


Now, we are going to find the suspicious shops with CR >= 3, then we will decide on the suspicious buyers from the shop

Since we are going to group the dataframes by shopid, we just need to define a function that does this for a single shopid

In [5]:
def get_sus(df):
  # Reset index for easy reference since the sorting made it untidy
  df = df.reset_index(drop=True)

  # Shop ID
  id = df["shopid"].iloc[0]

  # Number of rows
  n = len(df)

  # Create new column is_sus to check whether a specific order is suspicious
  # Set initially to False
  df["is_sus"] = False

  # For any two rows in the dataframe, consider all the orders in between, inclusively
  for i in range(n):
    for j in range(i,n):
      # Given the starting point i-th order, the j-th order is more than 1 hour past it, so we just ignore this and the following order
      if df.loc[j,"event_time"] - df.loc[i,"event_time"] > 3600:
        break
      
      # Otherwise, we have to check as follows :
      # Given i-th order and j-th order are not more than 1 hour apart
      # i == 0, meaning that the 1-hour interval started before the first order
      # j == n-1, meaning that it started after the final order
      # else, the interval started between (i-1)-th and i-th order and ends between j-th and (j+1)-th order
      if i == 0 or j == n-1 or df.loc[j+1,"event_time"] - df.loc[i-1,"event_time"] > 3600:
        order_1_hour = j-i+1
        unique_buyer_1_hour = df.loc[i:j,"userid"].nunique()
        # CR >= 3
        if order_1_hour >= 3*unique_buyer_1_hour:
          df.loc[i:j,"is_sus"] = True

  # Filter out the rows that are not suspicious
  df = df[df["is_sus"]]

  # Among the suspicious buyers, store the order frequency of each user to a dictionary
  frequency_dict = df["userid"].value_counts().to_dict()
  
  # Temporary list to store the most suspicious buyer
  temp = []
  max_freq = 0

  for k,v in sorted(frequency_dict.items()):
    # Found a new maximum, renew temp and insert k
    if max_freq < v:
      max_freq = v
      temp = [k]
    # Just add k to temp
    elif max_freq == v:
      temp.append(k)

  # Double check
  temp.sort()

  # Remember that we are doing this for a single shopid. Hence, the whole content of temp is indeed the most suspicious buyers for that shop
  output = '&'.join(list(map(str,temp)))

  # Add to the final dictionary
  if not output:
    output = '0'
  final_dict[id] = output


From the previous function, we need a final dictionary to contain the final result

In [6]:
final_dict = {}

Group the dataframe by shopid then apply the get_sus function for each shopid

In [7]:
train.groupby("shopid", as_index=False).apply(get_sus)

# Display top 3, no change from previous
train.head(3)

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,630720000
25985,31075200506752,104804492,97707522,630720000
81020,31075200506753,8715449,97707522,630720000


Nothing changed, but let's see what's final_dict

In [8]:
final_dict

{10009: '0',
 10051: '0',
 10061: '0',
 10084: '0',
 10100: '0',
 10107: '0',
 10108: '0',
 10110: '0',
 10132: '0',
 10133: '0',
 10151: '0',
 10153: '0',
 10155: '0',
 10159: '214988798',
 10166: '0',
 10178: '0',
 10186: '0',
 10195: '0',
 10208: '0',
 10214: '0',
 10217: '0',
 10253: '0',
 10281: '0',
 10282: '0',
 10287: '0',
 10311: '0',
 10324: '0',
 10345: '0',
 10346: '0',
 10352: '0',
 10353: '0',
 10358: '0',
 10359: '0',
 10360: '0',
 10363: '0',
 10375: '0',
 10378: '0',
 10380: '0',
 10388: '0',
 10396: '0',
 10402: '77819',
 10407: '0',
 10408: '0',
 10410: '0',
 10418: '0',
 10424: '0',
 10429: '0',
 10432: '0',
 10445: '0',
 10446: '0',
 10452: '0',
 10484: '0',
 10485: '0',
 10498: '0',
 10508: '0',
 10509: '0',
 10517: '0',
 10536: '672345',
 10546: '0',
 10555: '0',
 10556: '0',
 10557: '0',
 10563: '0',
 11518: '0',
 11608: '0',
 11613: '0',
 11671: '0',
 11722: '0',
 11752: '0',
 11918: '0',
 12568: '0',
 12581: '0',
 12713: '0',
 13238: '0',
 13405: '0',
 13511: 

Now that we have the dictionary, we can simply create a final dataframe!

In [9]:
# Replicate from train dataframe but just take unique shopids
final = pd.DataFrame({'shopid': train['shopid'].unique()})
final["userid"] = '0'

# Transfer all the data from final_dict
for i in range(len(final)):
  final.loc[i,"userid"] = final_dict[final.loc[i,"shopid"]]

final

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,9753706
3,190969466,0
4,2859407,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


And there we go! We have produced the final output

In [10]:
final.to_csv('submission.csv', index=False)